# BigQuery & Gemini AI: Chat Analysis Prototype

This notebook demonstrates how to use BigQuery and Gemini AI to analyze chat messages. It's designed to be run in a Kaggle or Google Colab environment.

## 1. Setup & Dependencies

First, let's install the necessary libraries.

In [ ]:
%pip install google-cloud-bigquery pandas pyarrow db-dtypes

## 2. Authentication

In [ ]:
from google.cloud import bigquery
import pandas as pd
import google.auth

# This will automatically find your default credentials.
# Make sure you have run `gcloud auth application-default login` locally.
credentials, project_id = google.auth.default()

# The project_id is detected automatically, but you can override it here
# project_id = "your-gcp-project-id"  # @param {type:"string"}
dataset_id = "your_bigquery_dataset"  # @param {type:"string"}
table_id = "chat_messages"  # @param {type:"string"}

client = bigquery.Client(credentials=credentials, project=project_id)

## 3. Load Sample Data

For this demo, we'll load a sample CSV of chat messages into a Pandas DataFrame and then upload it to BigQuery. In a real-world scenario, this data would likely be streamed directly into BigQuery.

In [ ]:
# Load the sample data from the CSV file included in the project
df = pd.read_csv('sample_chats.csv')

# Display the first few rows
print(df.head())

# Upload the DataFrame to BigQuery
table_ref = client.dataset(dataset_id).table(table_id)
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("message_id", "INTEGER"),
        bigquery.SchemaField("user_id", "STRING"),
        bigquery.SchemaField("timestamp", "TIMESTAMP"),
        bigquery.SchemaField("message", "STRING"),
    ],
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    df, table_ref, job_config=job_config
)
job.result()  # Wait for the job to complete

print(f"Loaded {job.output_rows} rows into {project_id}:{dataset_id}.{table_id}")

## 4. Analyze Messages with Gemini AI

Now, we'll use a BigQuery remote function with Gemini AI to analyze the chat messages for toxicity, sentiment, and topic. The following SQL query performs this analysis.

In [ ]:
sql = f"""
SELECT
  message_id,
  user_id,
  message,
  ml_generate_text_result['predictions'][0]['content'] AS analysis
FROM
  ML.GENERATE_TEXT(
    MODEL `{project_id}.{dataset_id}.gemini_model`,
    (
      SELECT
        message_id,
        user_id,
        message,
        CONCAT(
          'Analyze the following chat message for toxicity, sentiment, and topic. Return a JSON object with \"toxicity_score\" (1-10), \"sentiment\" (positive/negative/neutral), and \"topic\": ',
          message
        ) AS prompt
      FROM
        `{project_id}.{dataset_id}.{table_id}`
    ),
    STRUCT(
      0.2 AS temperature,
      1024 AS max_output_tokens
    )
  );
"""

analysis_df = client.query(sql).to_dataframe()
print(analysis_df.head())

## 5. Next Steps: Visualization

The `analysis_df` DataFrame now contains the original chat messages along with the AI-powered analysis. This data can be saved as a new BigQuery table and connected to Looker Studio for visualization.

See the `looker_studio_instructions.md` file for more details.